In [ ]:
import os

from dotenv import load_dotenv, dotenv_values 
# loading variables from .env file
load_dotenv() 

from openai import OpenAI
 
client = OpenAI()


assistantInstructions="""You are a resume grading assistant, and your job is to assess resumes for the position of GenAI Production Engineer at Altimetrik. You will be provided with a resume file, and and you must take plenty of time to read the entire file that you are given. If you are unable to read a resume, you should just say "I am unable to read the resume". You should never give an assessment for a resume you have not read in its entirety.
You will provide a score from 1 to 10, indicating how well the candidate fits this role. You will also give a short summary for why you have given this score.
You must follow this output format exactly. Do not give other information or describe your process for evaluation:
Score: [X out of 10]
Summary: [A short summary]
Use the following criteria to evaluate the candidate’s experience and skills against the job requirements, with a focus on both technical expertise and relevant industry experience.
Evaluation Criteria:
* Experience with AI/ML Production Systems: Does the candidate have at least 3 years of experience in software engineering with a specific focus on AI/ML project deployment? Look for experience with deploying or optimizing production-ready AI/ML systems.
* Technical Skills and Cloud Expertise: Does the candidate have hands-on experience with the required cloud and AI technologies (ChatGPT API, AWS, Azure, Snowflake, and Salesforce)? Also, assess familiarity with serverless and microservices architectures.
* Programming and DevOps Skills: Does the candidate demonstrate proficiency in Python, JavaScript, and TypeScript, as well as knowledge of DevOps tools (e.g., Git, Jenkins) and infrastructure as code (e.g., Terraform) and containerization tools (e.g., Docker, Kubernetes)?
* Experience with Retrieval-Augmented Generation: Does the candidate have specific experience building or optimizing RAG systems, including designing metadata schemas or using vector embeddings for improved data retrieval?
* Collaboration and Problem-Solving: Does the candidate’s experience show evidence of working in cross-functional teams and using strong problem-solving skills? Look for experience in agile environments and any collaborative projects in AI/ML contexts.
* Career Growth Potential and Cultural Fit: Does the candidate exhibit qualities that align with Altimetrik's innovative and growth-focused culture? Assess their motivation, as indicated by previous experience or in their cover letter, for growth, innovation, and working collaboratively in a fast-paced tech environment."""

#create a new assistant
assistant = client.beta.assistants.create(
    name="Resume Evaluation Assistant",
    instructions=assistantInstructions,
    model="gpt-4o",
    temperature=0.2,
    tools=[{"type": "file_search"}]
)

#select existing assistant
#my_assistant = client.beta.assistants.retrieve("asst_GLnbL3Ff5vfNMlKihTk8elws")


# Create a vector store called "GenAI Resumes"
vector_store = client.beta.vector_stores.create(name="GenAI Resumes")
 
# Ready the files for upload to OpenAI
filename="/Users/thomasgersic/Documents/Altimetrik/Recruiting/ResumesFromLinkedin/Resume.pdf"
file_paths = [filename]
file_streams = [open(path, "rb") for path in file_paths]
 
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

# Step 3: Update the assistant to use the new Vector Store
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

# Upload the user provided file to OpenAI
message_file = client.files.create(
  file=open(filename, "rb"), purpose="assistants"
)
 
# Create a thread and attach the file to the message
thread = client.beta.threads.create(
    messages=[
    {
        "role": "user",
        "content": "Evaluate this resume",
        # Attach the new file to the message.
        "attachments": [
            { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
        ],
    }
    ]
)
 
# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)

from typing_extensions import override
from openai import AssistantEventHandler, OpenAI
 
client = OpenAI()

#Reference: https://platform.openai.com/docs/api-reference/assistants-streaming/events
class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)

    @override
    def on_message_done(self, message) -> None:
        # print a citation to the file searched
        message_content = message.content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(
                annotation.text, f"[{index}]"
            )
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        print(message_content.value)
        print("\n".join(citations))
        
    @override
    #reference: https://github.com/openai/openai-python/blob/main/helpers.md
    def on_end(self):
        #cleanup
        response = client.beta.assistants.delete(assistant.id)
        print(response)
    


# Then, we use the stream SDK helper
# with the EventHandler class to create the Run
# and stream the response.

with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    event_handler=EventHandler(),
    temperature=0.2
) as stream:
    stream.until_done()


completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)
ToolResourcesFileSearch(vector_store_ids=['vs_XPY9In8GPRHjeWbhVoqMfiPB'])

assistant > file_search


assistant > Score: 7 out of 10

Summary: The candidate has a solid background in AI/ML research and DevOps, with over five years of experience, including leading a research team and implementing neural network quantization techniques. They possess relevant certifications in AWS and have experience with CI/CD pipelines, Docker, Kubernetes, and various AI/ML tools like TensorFlow and PyTorch. However, the resume lacks specific experience with some of the required technologies such as ChatGPT API, Azure, Snowflake, and Salesforce, and there is no mention of experience with Retrieval-Augmented Generation systems. The candidate demonstrates strong collaboration and problem-solving skills, which align with the role's requirements[0].
[0] Resume.pdf
AssistantDeleted(id='asst_HRhlBc0sVX2tb07XxW4A4YSo', deleted=True